In [1]:
!pip install Ipython

import numpy as np
import seaborn as sns 
import pandas as pd
from tqdm import tqdm
import cv2
from tensorflow.keras.applications import Xception
from tensorflow.keras.utils import plot_model

In [2]:
TRAIN_PATH = '../input/severstal-steel-defect-detection'
TRAIN_IMAGES = '../input/severstal-steel-defect-detection/train_images'
TEST_IMAGES = '../input/severstal-steel-defect-detection/test_images'

In [3]:
df_train = pd.read_csv(TRAIN_PATH + '/train.csv')
df_submission = pd.read_csv(TRAIN_PATH + '/sample_submission.csv')

In [4]:
df_train.head()

,ImageId,ClassId,EncodedPixels
0,0002cc93b.jpg,1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0007a71bf.jpg,3,18661 28 18863 82 19091 110 19347 110 19603 11...
2,000a4bcdd.jpg,1,37607 3 37858 8 38108 14 38359 20 38610 25 388...
3,000f6bf48.jpg,4,131973 1 132228 4 132483 6 132738 8 132993 11 ...
4,0014fce06.jpg,3,229501 11 229741 33 229981 55 230221 77 230468...


In [5]:
df_train.shape

(7095, 3)

In [6]:
df_train = df_train.iloc[0: 1000]

In [7]:
import matplotlib.pyplot as plt

In [8]:
img = plt.imread(TRAIN_IMAGES + '/0002cc93b.jpg')

In [9]:
img

array([[[ 70,  70,  70],
        [ 70,  70,  70],
        [ 68,  68,  68],
        ...,
        [ 48,  48,  48],
        [ 48,  48,  48],
        [ 50,  50,  50]],

       [[ 66,  66,  66],
        [ 68,  68,  68],
        [ 68,  68,  68],
        ...,
        [ 48,  48,  48],
        [ 49,  49,  49],
        [ 51,  51,  51]],

       [[ 61,  61,  61],
        [ 64,  64,  64],
        [ 65,  65,  65],
        ...,
        [ 49,  49,  49],
        [ 51,  51,  51],
        [ 54,  54,  54]],

       ...,

       [[155, 155, 155],
        [133, 133, 133],
        [131, 131, 131],
        ...,
        [ 51,  51,  51],
        [ 51,  51,  51],
        [ 50,  50,  50]],

       [[160, 160, 160],
        [111, 111, 111],
        [100, 100, 100],
        ...,
        [ 55,  55,  55],
        [ 54,  54,  54],
        [ 48,  48,  48]],

       [[155, 155, 155],
        [114, 114, 114],
        [ 98,  98,  98],
        ...,
        [ 58,  58,  58],
        [ 58,  58,  58],
        [ 50,  50,  50]]

In [10]:
df_train.head()

,ImageId,ClassId,EncodedPixels
0,0002cc93b.jpg,1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0007a71bf.jpg,3,18661 28 18863 82 19091 110 19347 110 19603 11...
2,000a4bcdd.jpg,1,37607 3 37858 8 38108 14 38359 20 38610 25 388...
3,000f6bf48.jpg,4,131973 1 132228 4 132483 6 132738 8 132993 11 ...
4,0014fce06.jpg,3,229501 11 229741 33 229981 55 230221 77 230468...


In [11]:
images = []
classID = []
for image_id, class_id, encoded_bits in tqdm(df_train.values):
    img = cv2.imread(TRAIN_IMAGES + '/{}'.format(image_id), cv2.IMREAD_COLOR)
    img = img / 255
    images.append(img)
    classID.append(class_id)

100%|██████████| 1000/1000 [00:17<00:00, 56.03it/s]


In [12]:
images[0]

array([[[0.2745098 , 0.2745098 , 0.2745098 ],
        [0.2745098 , 0.2745098 , 0.2745098 ],
        [0.26666667, 0.26666667, 0.26666667],
        ...,
        [0.18823529, 0.18823529, 0.18823529],
        [0.18823529, 0.18823529, 0.18823529],
        [0.19607843, 0.19607843, 0.19607843]],

       [[0.25882353, 0.25882353, 0.25882353],
        [0.26666667, 0.26666667, 0.26666667],
        [0.26666667, 0.26666667, 0.26666667],
        ...,
        [0.18823529, 0.18823529, 0.18823529],
        [0.19215686, 0.19215686, 0.19215686],
        [0.2       , 0.2       , 0.2       ]],

       [[0.23921569, 0.23921569, 0.23921569],
        [0.25098039, 0.25098039, 0.25098039],
        [0.25490196, 0.25490196, 0.25490196],
        ...,
        [0.19215686, 0.19215686, 0.19215686],
        [0.2       , 0.2       , 0.2       ],
        [0.21176471, 0.21176471, 0.21176471]],

       ...,

       [[0.60784314, 0.60784314, 0.60784314],
        [0.52156863, 0.52156863, 0.52156863],
        [0.51372549, 0

In [13]:
len(classID)

1000

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
df_train.head()

,ImageId,ClassId,EncodedPixels
0,0002cc93b.jpg,1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0007a71bf.jpg,3,18661 28 18863 82 19091 110 19347 110 19603 11...
2,000a4bcdd.jpg,1,37607 3 37858 8 38108 14 38359 20 38610 25 388...
3,000f6bf48.jpg,4,131973 1 132228 4 132483 6 132738 8 132993 11 ...
4,0014fce06.jpg,3,229501 11 229741 33 229981 55 230221 77 230468...


In [16]:
df_train['Converted_pixels'] = images

In [17]:
X = images
y = df_train['ClassId'].values

In [18]:
len(images)

1000

In [19]:
from sklearn.preprocessing import LabelEncoder 

In [20]:
encoder = LabelEncoder()
y_converted = encoder.fit_transform(y)

In [21]:
from tensorflow.keras.utils import to_categorical

In [22]:
y_final = to_categorical(y_converted)

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
X_train, X_cv, y_train, y_cv = train_test_split(X, y_final, random_state = 50, test_size = 0.3)

In [25]:
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential

In [26]:
model = Sequential()
model.add(Dense(100, activation = 'relu'))
model.add(Dense(50, activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(4, activation = 'softmax'))

model.compile(optimizer = 'adam', )

In [ ]:
model.fit(X_train, y_train, epochs = 10, validation_data = (X_cv, y_cv))

In [ ]:
import cv2
import matplotlib.pyplot as plt

In [ ]:
img = plt.imread(train_path + '/{}'.format(images[0]))
plt.imshow(img)
plt.show()

In [ ]:
img.shape

In [ ]:
1600 * 3

In [ ]:
4800 / 69

In [ ]:
np.sqrt(4800)

In [ ]:
sample_submission = pd.read_csv('../input/severstal-steel-defect-detection/sample_submission.csv')

In [ ]:
sample_submission.head()

In [ ]:
df_train.head()

In [ ]:
np.array(df_train['EncodedPixels'].iloc[0])

In [ ]:
df_train.head()